#**Deep Learning Project-2**

##Submitted by :
##Name: Shweta Sharma
##UIN: 433003780

Project two. Due: 11:59pm (Central Time) on Friday 4/7/2023. (Submit all your code with detailed/clear explanations in a single Jupyter notebook file in Canvas. Email your trained model AS WELL AS its output for the "testing dataset" to csce636projects2023@gmail.com with your name and UIN . 105 points)

This project is on machine translation for two "artificial" languages: an "Input Language" and an "Output Language". We want to build a model to translate the texts in the "Input Language" to texts in the "Output Language". For example, a text in the "Input Language" can be "a g b f a f a e a k a j c f b f c d a k a k c e b g a h a k b d b f b f b d c d " , and its translation to the "Output Language" is "b f c f b f c d a j e f g c e b g a k i j b d b f a k l m b f b d a h ed ee ef a k k eg a k h eh a e ei c d a f ej ek a g d el".

As a training dataset, a list of 5,000 texts in the Input Language are here, and the corresponding list of 5,000 texts in the Output Language are here.  (After downloading them, you can use pickle.load(open(path,'rb')) to open them. They are two lists of strings in Python.)

Your task is to design and train a good model, which can take texts in the "Input language" as its input, translate them to the corresponding texts in the "Output Language" and output them. (Note: the final output of your code should not include extra tokens such as [start] or [end]. So if your model's output includes such extra tokens, you need to write a few more lines of code to remove them.)

Your trained model will be evaluated using a testing set (which is shown below, and is similar to the above training set). Its performance will be measured by its "accuracy for words", defined as follows. Given a text, a word is a continuous sequence of letters separated by space in the text. (For example, the text "b f c ej ek d el" has 7 words: "b", "f", "c", "ej", "ek", "d" and "el".) "Accuracy" is defined as the fraction of words in the ground-truth text (in the Output Language) that are shown correctly in your model's output. (For example, if the ground-truth text is "b f c ej ek d el" and your model's output is "b c el", then your result has 1 correct word and 6 mistaken words (since the ground-truth text has 7 words in total). If your model's output is "b c el ej ek a b c d f g", then your result has 3 correct words and 4 mistaken words.)

You should submit THREE things:

**1) Place all your code in a single Jupyter notebook, and submit it in Canvas. You should accompany your code with clear/detailed explanations, so that we can understand the methods you used.**

**2) Email your trained model to csce636projects2023@gmail.com. (Please make sure to include your name and UIN in the email.) We need to be able to test your model very easily in Google CoLab by running a simple line of code on our test set (described below). Your model should accept a batch of sentences (not just a single sentence) as input, and output the corresponding batch of "Output Language" sentences. You need to specify in your Jupyter notebook what that single line of code is that can do the above.**

**3) As the test dataset for this project, a list of 5,000 texts in the Input Language are here. Please run your trained model to get the 5,000  corresponding texts in the Output Language (as a list of 5,000 strings in Python), and email them to csce636projects2023@gmail.com. (You can use pickle.dump(yourList, open(path,'wb')) to save your list in a file, and then attach it to your email.) Please note that we will use your model to verify the above submitted texts, but we will use our own code to compute the accuracy of your model on the test dataset.**

Method of grading: your grade for this project will be equal to:

(test_accuracy_for_words +  0.05) x 100

For example, if your test accuracy is 0.75, then the grade is 80; and if your test accuracy is 0.98, then the grade is 103. However, if the code in your Jupyter notebook is incomplete, the grade will be 0; if the explanations in the Jupyter notebook are not clear, then 5 points will be taken away; and if we need to make any (even simple) modification to your code in order to run it for testing (instead of simply running a single line of code following your instruction), then some points will be taken away (depending on how serious the modification needs to be).

# **Solution:**

Loading the Training Inputs and Outputs using pickle:

In [ ]:
import pickle
train_input_path="/content/DS_5_train_input"
train_output_path="/content/DS_5_train_output"

train_inputs=pickle.load(open(train_input_path, 'rb'))
train_outputs=pickle.load(open(train_output_path, 'rb'))

In [ ]:
len(train_inputs)

5000

In [ ]:
len(train_outputs)

5000

In [ ]:
res = max(train_inputs, key = len)
print(len(res.split()))

64


In [ ]:
res = max(train_outputs, key = len)
print(len(res.split()))

95


In [ ]:
print(train_inputs[110])
print(train_outputs[110])

a f a j a f c f c e a e b g a h a f b e c f a g a f b f b f c d c g a d b e b f 
c f c e a f d e b g a e g b e c f a f i j b f b f a f l m c d a g ed ee c g a h k ef eg a j f h eh b e b f a d ej ek a f ei el 


This below code checks the unique words present in the vocabulary of Training inputs and Training Outputs which comes out to be 34, which we can use as the vocab_size for our Deep Learning model.

In [ ]:
vocab1 = set()
for i in range(len(train_inputs)):
  for j in train_inputs[i].split():
    vocab1.add(j)

print(len(vocab1))

vocab2=set()
for i in range(len(train_outputs)):
  for j in train_outputs[i].split():
    vocab2.add(j)

print(len(vocab2))

11
34


In [ ]:
text_pairs=[]
for i in range(len(train_inputs)):
  input = train_inputs[i]
  output = "[start] " + train_outputs[i] + " [end]"
  text_pairs.append((input,output))

In [ ]:
import random
print(random.choice(text_pairs))

('a h a d c f b d a e a k c g a f c e a f a h c f b d a f b d c d c e a g c f c g ', '[start] c f b d a d d e c g c e c f b d b d c d a f k l a h i j m c e a f ed ee a f h ef a k g eg a e eh c f c g a g ej ek a h f ei el  [end]')


This code shuffles a list of text_pairs, splits it into training and validation sets, and determines the number of samples for each set.

The random.shuffle() function is used to shuffle the text_pairs list randomly. This is often done to ensure that the order of the data does not bias the learning algorithm.

The code then calculates the number of validation samples (num_val_samples) as 15% of the total number of text pairs in the list. It also calculates the number of training samples (num_train_samples) as the difference between the total number of text pairs and twice the number of validation samples.

The train_pairs variable is then assigned the first num_train_samples pairs in the shuffled list, and val_pairs is assigned the next num_val_samples pairs.

These two sets can then be used for training and validation of a machine learning model, respectively.

In [ ]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]

**Preparing datasets for the translation task**

**target_vectorization  and  source_vectorization:** These lines of code define two TextVectorization layers in TensorFlow. TextVectorization is a layer that takes in text as input and converts it into a sequence of integers, where each integer represents a word in the vocabulary.

source_vectorization is used to vectorize the input text data, and target_vectorization is used to vectorize the target text data. The max_tokens parameter specifies the maximum vocabulary size to consider. Any words outside of this vocabulary will be treated as an out-of-vocabulary token.

output_mode specifies the output type of the layer, which is set to "int" in this case to output integer sequences. output_sequence_length specifies the length of the output sequence, which is set to sequence_length for source_vectorization and sequence_length + 1 for target_vectorization. The standardize parameter is used to apply a custom function to preprocess the text before vectorization.

Overall, these layers are useful for preparing text data for use in deep learning models, such as natural language processing (NLP) models.


**make_dataset()** is a Python function that creates a TensorFlow dataset from a list of input-output text pairs.

First, the function separates the input and output texts into separate lists. Then, it creates a tf.data.Dataset object using the from_tensor_slices() method, which creates a dataset with one element for each input-output pair.

Next, the dataset is batched using the batch() method, which combines individual elements into batches of a specified size (batch_size). The map() method is then used to apply the format_dataset function to each element of the dataset in parallel (num_parallel_calls=4). The format_dataset function likely performs some preprocessing on the input-output pairs, such as tokenization or padding.

Finally, the dataset is shuffled (shuffle()), cached (cache()), and prefetched (prefetch()) for optimized performance during training.

The returned dataset object can then be used to train a machine learning model, typically a sequence-to-sequence model for tasks such as machine translation or text summarization.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import string
import re

vocab_size = 35 #since total vocabulary size is 34
sequence_length = 100 #since maximum length currently for output is 95

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

train_input_texts = [pair[0] for pair in train_pairs]
train_output_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_input_texts)
target_vectorization.adapt(train_output_texts)

batch_size = 32

def format_dataset(inp, out):
    inp = source_vectorization(inp)
    out = target_vectorization(out)
    return ({
        "input": inp,
        "output": out[:, :-1],
    }, out[:,1:])

def make_dataset(pairs):
    input_texts, output_texts = zip(*pairs)
    input_texts = list(input_texts)
    output_texts = list(output_texts)
    dataset = tf.data.Dataset.from_tensor_slices((input_texts, output_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
train_ds.take(1)

<_TakeDataset element_spec=({'input': TensorSpec(shape=(None, 100), dtype=tf.int64, name=None), 'output': TensorSpec(shape=(None, 100), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 100), dtype=tf.int64, name=None))>

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['input'].shape: {inputs['input'].shape}")
    print(f"inputs['output'].shape: {inputs['output'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['input'].shape: (32, 100)
inputs['output'].shape: (32, 100)
targets.shape: (32, 100)


### Sequence-to-sequence learning with Transformer

#### The Transformer Encoder and Decoder

**The `TransformerEncoder`**

This is a class definition for a single layer in a transformer encoder. A transformer is a type of neural network architecture commonly used in natural language processing tasks, such as machine translation or text generation.

The TransformerEncoder class takes in several parameters that define the layer's properties.

embed_dim specifies the dimensionality of the input and output embeddings, which are used to represent words or tokens in the input sequence.

dense_dim specifies the number of units in the dense layer of the feedforward network, which is used to process the output of the self-attention mechanism.

num_heads specifies the number of heads in the multi-head attention layer. Multi-head attention enables the model to attend to different positions in the input sequence simultaneously, which can improve performance on certain tasks.

hidden_layer specifies the number of units in the final hidden layer of the feedforward network.

dropout_prob specifies the probability of dropout regularization being applied during training.

The call method defines the computation performed by the layer. The layer first applies a multi-head attention mechanism to the input sequence, using layers.MultiHeadAttention. The output of the attention mechanism is then passed through a dense feedforward network with several layers, using layers.Dense. Layer normalization is applied after the attention and dense layers, using layers.LayerNormalization. Finally, the output of the dense feedforward network is added to the original input and normalized again to produce the layer's final output.

This class definition can be used to create multiple transformer encoder layers in a larger model, which can be trained on natural language processing tasks.


In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, hidden_layer, dropout_prob, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.hidden_layer = hidden_layer
        self.dropout_prob = dropout_prob
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dropout(dropout_prob),
             layers.Dense(dense_dim, activation="relu"),
             layers.Dropout(dropout_prob),
             layers.Dense(hidden_layer, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
            "hidden_layer": self.hidden_layer,
            "dropout_prob": self.dropout_prob,
        })
        return config

**The `TransformerDecoder`**

This is a class definition for a Transformer Decoder in TensorFlow. The Transformer Decoder is a crucial component of a Transformer model, which is a popular deep learning architecture used for natural language processing tasks.

The Transformer Decoder takes in encoded inputs and generates a sequence of tokens one by one. The input to the decoder is a concatenation of the encoded input and the previous tokens generated by the decoder, shifted one position to the right (i.e., the decoder tries to predict the next token given the previous tokens).

The decoder contains two Multi-Head Attention layers, which attend to the encoded input and the previous tokens, respectively. The output of the Multi-Head Attention layers is concatenated with the previous tokens and passed through a series of dense layers to generate the final output token.

The get_causal_attention_mask method returns a mask matrix that prevents the decoder from attending to future tokens during training. This is achieved by setting all the entries in the upper triangular part of the mask to zero.

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, hidden_layer, dropout_prob, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.hidden_layer = hidden_layer
        self.dropout_prob = dropout_prob
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dropout(dropout_prob),
             layers.Dense(dense_dim, activation="relu"),
             layers.Dropout(dropout_prob),
             layers.Dense(hidden_layer, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
            "hidden_layer": self.hidden_layer,
            "dropout_prob": self.dropout_prob,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

#### Putting it all together: A Transformer for machine translation

**PositionalEmbedding layer**

The PositionalEmbedding layer is a custom layer in a neural network that adds positional information to the input embeddings. It takes three parameters: sequence_length, input_dim, and output_dim. The sequence_length is the length of the input sequence, the input_dim is the dimensionality of the input embeddings, and the output_dim is the dimensionality of the positional embeddings.

In the __init__ method, the layer initializes two Embedding layers: token_embeddings and position_embeddings. The token_embeddings layer learns embeddings for the input tokens, and the position_embeddings layer learns embeddings for the positions of the tokens in the input sequence.

In the call method, the layer takes the input tokens and calculates the positions of the tokens. It then embeds the input tokens and the positions separately, and adds the two embeddings together.

The compute_mask method returns a mask for the input sequence, which is used to ignore padding tokens in the input sequence during training.

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**End-to-end Transformer**

We are defining a transformer model for sequence to sequence tasks with the encoder-decoder architecture. The input to the model is a pair of sequences, an input sequence, and an output sequence, and the model generates the output sequence given the input sequence.

The encoder_inputs and decoder_inputs are input layers that will accept input sequences of integers. The vocab_size specifies the size of the vocabulary for the input sequences. The PositionalEmbedding layer is used to add positional information to the input sequences.

The TransformerEncoder layer takes the embedded input sequence and applies the encoder component of the transformer architecture, producing the encoded sequence.

The TransformerDecoder layer applies the decoder component of the transformer architecture to the embedded output sequence, using the encoded sequence as context, and generates the output sequence.

Finally, the output is passed through a Dense layer with a softmax activation function to get the probability distribution over the vocabulary for each output token.

Overall, this model is designed to be used for tasks such as machine translation, where the input sequence is in one language, and the output sequence is in another language.






In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
hidden_layer = 512
dropout_prob = 0.5

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="input")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
#x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads, hidden_layer, dropout_prob)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="output")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads, hidden_layer, dropout_prob)(x, encoder_outputs)
#x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

**Training the sequence-to-sequence Transformer**

The compile function specifies the optimizer, loss function, and metrics to be used during training. In this case, you have chosen the rmsprop optimizer, sparse_categorical_crossentropy as the loss function, and accuracy as the evaluation metric.

The fit function trains the model using the training data (train_ds), for a specified number of epochs (250 in this case). You have also specified a validation dataset (val_ds) to evaluate the model's performance on during training. Additionally, you have specified a callback function (ModelCheckpoint) that will save the model weights whenever the validation loss improves.

Once the model is trained, the weights will be saved to a file named "transformer_model.keras" in the current directory.

In [ ]:

callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_model.keras",
                                    save_best_only=True)
]

transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=250, validation_data=val_ds, callbacks=callbacks) #epochs=220

Epoch 1/250
110/110 [==============================] - 25s 142ms/step - loss: 2.7314 - accuracy: 0.2215 - val_loss: 1.7469 - val_accuracy: 0.3498
Epoch 2/250
110/110 [==============================] - 15s 134ms/step - loss: 1.7563 - accuracy: 0.3415 - val_loss: 1.5134 - val_accuracy: 0.4070
Epoch 3/250
110/110 [==============================] - 15s 136ms/step - loss: 1.6200 - accuracy: 0.3760 - val_loss: 1.4570 - val_accuracy: 0.4212
Epoch 4/250
110/110 [==============================] - 15s 135ms/step - loss: 1.5366 - accuracy: 0.3969 - val_loss: 1.4674 - val_accuracy: 0.4186
Epoch 5/250
110/110 [==============================] - 15s 141ms/step - loss: 1.4846 - accuracy: 0.4127 - val_loss: 1.3709 - val_accuracy: 0.4480
Epoch 6/250
110/110 [==============================] - 15s 139ms/step - loss: 1.4440 - accuracy: 0.4250 - val_loss: 1.4232 - val_accuracy: 0.4217
Epoch 7/250
110/110 [==============================] - 15s 140ms/step - loss: 1.4074 - accuracy: 0.4377 - val_loss: 1.3595 -

**Translating new sentences with our Transformer model**

This code defines a custom objects dictionary containing the PositionalEmbedding, TransformerEncoder, and TransformerDecoder layers used in a previously trained Keras model saved in the file "transformer_model.keras". It then loads the model from memory using the keras.models.load_model() method with the custom_object_scope() function to ensure that the custom objects are properly reconstructed.

Note that this assumes that the necessary modules and data have been previously loaded into memory, and that the "transformer_model.keras" file is in the current working directory.

In [ ]:
import pickle
from keras.utils import custom_object_scope


# Define the custom objects dictionary
custom_objects = {'PositionalEmbedding': PositionalEmbedding, 'TransformerEncoder': TransformerEncoder, 'TransformerDecoder': TransformerDecoder}

# Load the model using custom_object_scope
with custom_object_scope(custom_objects):
    model_from_memory = keras.models.load_model("transformer_model.keras")

The decode_sequence function takes an input sentence in the source language as a string and returns the corresponding translated sentence in the target language.

Here is how the function works:

*  Tokenize the input sentence using the source_vectorization object.
* Initialize the decoded sentence with the [start] token.
*Loop over the possible target tokens up to the maximum length of the decoded sentence.
*Tokenize the current decoded sentence using the target_vectorization object.
*Feed the tokenized input sentence and tokenized decoded sentence into the model_from_memory model to get the predictions for the next target token.
*Select the target token with the highest probability from the predictions.
*Add the selected target token to the decoded sentence.
If the selected target token is the [end] token, stop looping and return the decoded sentence.





In [ ]:

import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 100

test_input_path="/content/DS_5_test_input"
test_inputs=pickle.load(open(test_input_path, 'rb'))

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = model_from_memory(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_outputs = []
for i in range(len(test_inputs)): #change it
    input_sentence = test_inputs[i]
    output = decode_sequence(input_sentence)
    output = output[8:-6]
    test_outputs.append(output)




KeyboardInterrupt: ignored

In [ ]:
print(len(test_outputs))

3483


In [ ]:
test_output_path="/content/DS_5_test_output"
pickle.dump(test_outputs, open(test_output_path,'wb'))